# Import Libraries

In [52]:
import pandas as pd
import numpy as np

#Use Keras Neural Network Components. This will be used to construct the LSTM
from keras import backend as K
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dense, Dropout
from keras.layers import BatchNormalization
from keras.regularizers import L1L2

#Import Tensorflow 
import tensorflow as tf

# Ensure Reproducability of Results

In [53]:
np.random.seed(42)
tf.random.set_seed(42)

# Fetch and Set Up Data

In [54]:
#Numerically Encode the Activities
ACTIVITIES = {
    0: 'WALKING',
    1: 'WALKING_UPSTAIRS',
    2: 'WALKING_DOWNSTAIRS',
    3: 'SITTING',
    4: 'STANDING',
    5: 'LAYING',
}

In [55]:
DATDIR = "data/UCI HAR Dataset"
SIGNALS = [
    "body_acc_x",
    "body_acc_y",
    "body_acc_z",
    "body_gyro_x",
    "body_gyro_y",
    "body_gyro_z",
    "total_acc_x",
    "total_acc_y",
    "total_acc_z"
    ]

In [56]:
#train_test: 0 if we want to fetch the training signal data, 1 if we want to fetch the test signal data
def fetch_raw_signal_data(train_test, signal):
    file_path = DATDIR + f"/{'train' if train_test == 0 else 'test'}/Inertial Signals/{signal}_{'train' if train_test == 0 else 'test'}.txt"
    return pd.read_csv(file_path, delim_whitespace = True, header = None)

#Same purpose as previous function but, in this case, we are fetching ALL the raw signal data!
#train_test: 0 if we want to fetch the training signal data, 1 if we want to fetch the test signal data
def fetch_all_raw_signals(train_test):
    data_raw_signals = []
    for signal in SIGNALS:
        raw_signal_data = fetch_raw_signal_data(train_test, signal).to_numpy()
        data_raw_signals.append(raw_signal_data)
    
    data_raw_signals = np.array(data_raw_signals)
    return np.transpose(data_raw_signals, axes = (1, 2, 0))

#Fetch Labels
#train_test: 0 if we want to fetch the training signal data, 1 if we want to fetch the test signal data
def fetch_labels(train_test):
    file_path = DATDIR + f"/{'train' if train_test == 0 else 'test'}/y_{'train' if train_test == 0 else 'test'}.txt"
    return pd.get_dummies(pd.read_csv(file_path, delim_whitespace=True, header = None)[0]).to_numpy()

In [57]:
X_train, X_test, Y_train, Y_test = fetch_all_raw_signals(0), fetch_all_raw_signals(1), fetch_labels(0), fetch_labels(1)

N = X_train.shape[0]
T = X_train.shape[1]
D = X_train.shape[2]

print(N, T, D)

n_classes = Y_train.shape[1]
print(n_classes)

7352 128 9
6


# Define LSTM Architecture

## Use 1 LSTM Layer

In [58]:
##Hyperparameters
epochs = 30
batch_size = 16
n_hidden = 32

In [59]:
# Initiliazing the sequential model
model = Sequential()

# Configuring the parameters
model.add(LSTM(n_hidden, input_shape = (T, D)))

# Adding a dropout layer
model.add(Dropout(0.5))

# Adding a dense output layer with sigmoid activation
model.add(Dense(n_classes, activation = 'softmax'))

model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_2 (LSTM)               (None, 32)                5376      
                                                                 
 dropout_2 (Dropout)         (None, 32)                0         
                                                                 
 dense_2 (Dense)             (None, 6)                 198       
                                                                 
Total params: 5574 (21.77 KB)
Trainable params: 5574 (21.77 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [60]:
# Compiling the model
model.compile(loss='categorical_crossentropy', optimizer='rmsprop', metrics=['accuracy'])

In [61]:
# Training the model
model.fit(X_train, Y_train, batch_size=batch_size, validation_data=(X_test, Y_test),epochs=epochs)

Epoch 1/30
460/460 [==============================] - 12s 22ms/step - loss: 1.2004 - accuracy: 0.5065 - val_loss: 0.9679 - val_accuracy: 0.5721
Epoch 2/30
460/460 [==============================] - 12s 26ms/step - loss: 0.7701 - accuracy: 0.6780 - val_loss: 0.7493 - val_accuracy: 0.7363
Epoch 3/30
460/460 [==============================] - 10s 22ms/step - loss: 0.5696 - accuracy: 0.7825 - val_loss: 0.7354 - val_accuracy: 0.7503
Epoch 4/30
460/460 [==============================] - 11s 23ms/step - loss: 0.4394 - accuracy: 0.8451 - val_loss: 0.5599 - val_accuracy: 0.8168
Epoch 5/30
460/460 [==============================] - 11s 23ms/step - loss: 0.3509 - accuracy: 0.8860 - val_loss: 0.5394 - val_accuracy: 0.8331
Epoch 6/30
460/460 [==============================] - 11s 24ms/step - loss: 0.3169 - accuracy: 0.9018 - val_loss: 0.5055 - val_accuracy: 0.8476
Epoch 7/30
460/460 [==============================] - 11s 23ms/step - loss: 0.2831 - accuracy: 0.9082 - val_loss: 0.3913 - val_accuracy:

In [62]:
def generate_confusion_matrix(Y_true, Y_pred):
    Y_true = pd.Series([ACTIVITIES[y] for y in np.argmax(Y_true, axis=1)])
    Y_pred = pd.Series([ACTIVITIES[y] for y in np.argmax(Y_pred, axis=1)])
    return pd.crosstab(Y_true, Y_pred, rownames=['True'], colnames=['Pred'])

In [63]:
#Generate Confusion Matrix
generate_confusion_matrix(Y_test, model.predict(X_test))

93/93 [==============================] - 1s 7ms/step


Pred,LAYING,SITTING,STANDING,WALKING,WALKING_DOWNSTAIRS,WALKING_UPSTAIRS
True,,,,,,
LAYING,534,2,0,0,0,1
SITTING,0,373,101,0,17,0
STANDING,0,101,427,1,3,0
WALKING,0,2,0,476,5,13
WALKING_DOWNSTAIRS,0,0,0,22,384,14
WALKING_UPSTAIRS,0,2,3,11,1,454


In [64]:
score = model.evaluate(X_test, Y_test)

print("\n   cat_crossentropy  ||   accuracy ")
print("  ____________________________________")
print(score)

93/93 [==============================] - 1s 7ms/step - loss: 0.4454 - accuracy: 0.8985

   cat_crossentropy  ||   accuracy 
  ____________________________________
[0.4454444348812103, 0.8985409140586853]


## Use 2 LSTM Layers

In [65]:
##Hyperparameters
epochs = 30
batch_size = 16
n_hidden = 32

In [67]:
#Define Model Architecture
model = Sequential()
model.add(LSTM(48, input_shape=(T, D), return_sequences=True))
model.add(BatchNormalization())
model.add(Dropout(0.50))

model.add(LSTM(32))
model.add(Dropout(0.50))
model.add(Dense(n_classes, activation = 'softmax'))

model.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_5 (LSTM)               (None, 128, 48)           11136     
                                                                 
 batch_normalization_1 (Bat  (None, 128, 48)           192       
 chNormalization)                                                
                                                                 
 dropout_4 (Dropout)         (None, 128, 48)           0         
                                                                 
 lstm_6 (LSTM)               (None, 32)                10368     
                                                                 
 dropout_5 (Dropout)         (None, 32)                0         
                                                                 
 dense_3 (Dense)             (None, 6)                 198       
                                                      

In [68]:
# Compiling the model
model.compile(loss='categorical_crossentropy', optimizer='rmsprop', metrics=['accuracy'])

In [69]:
# Training the model
model.fit(X_train, Y_train, batch_size=batch_size, validation_data=(X_test, Y_test),epochs=epochs)

Epoch 1/30
460/460 [==============================] - 36s 68ms/step - loss: 0.7896 - accuracy: 0.7063 - val_loss: 1.0415 - val_accuracy: 0.6590
Epoch 2/30
460/460 [==============================] - 29s 62ms/step - loss: 0.3666 - accuracy: 0.8845 - val_loss: 0.3294 - val_accuracy: 0.8836
Epoch 3/30
460/460 [==============================] - 29s 64ms/step - loss: 0.2461 - accuracy: 0.9196 - val_loss: 0.3975 - val_accuracy: 0.8636
Epoch 4/30
460/460 [==============================] - 30s 66ms/step - loss: 0.2048 - accuracy: 0.9335 - val_loss: 0.2371 - val_accuracy: 0.9118
Epoch 5/30
460/460 [==============================] - 29s 64ms/step - loss: 0.1937 - accuracy: 0.9343 - val_loss: 0.2552 - val_accuracy: 0.9165
Epoch 6/30
460/460 [==============================] - 30s 64ms/step - loss: 0.1743 - accuracy: 0.9387 - val_loss: 0.2341 - val_accuracy: 0.9165
Epoch 7/30
460/460 [==============================] - 30s 66ms/step - loss: 0.1638 - accuracy: 0.9392 - val_loss: 0.2282 - val_accuracy:

In [70]:
#Generate Confusion Matrix
generate_confusion_matrix(Y_test, model.predict(X_test))

93/93 [==============================] - 3s 19ms/step


Pred,LAYING,SITTING,STANDING,WALKING,WALKING_DOWNSTAIRS,WALKING_UPSTAIRS
True,,,,,,
LAYING,537,0,0,0,0,0
SITTING,0,409,82,0,0,0
STANDING,0,95,437,0,0,0
WALKING,0,0,0,472,24,0
WALKING_DOWNSTAIRS,0,0,0,3,417,0
WALKING_UPSTAIRS,0,0,5,2,10,454


In [71]:
score = model.evaluate(X_test, Y_test)

print("\n   cat_crossentropy  ||   accuracy ")
print("  ____________________________________")
print(score)

93/93 [==============================] - 2s 21ms/step - loss: 0.3203 - accuracy: 0.9250

   cat_crossentropy  ||   accuracy 
  ____________________________________
[0.32028326392173767, 0.9250084757804871]
